### Importe

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parents[0]))
import config

# ==================== KONFIGURATION ====================


2025-09-19 01:57:12.695 | INFO     | config:<module>:11 - PROJ_ROOT path is: E:\Coding\Python\MachineLearning_SC


In [2]:
print("Python Version:", sys.version)
print("TensorFlow Version:", tf.__version__)
print("CUDA verfügbar:", tf.test.is_built_with_cuda())
print("GPUs erkannt:", tf.config.list_physical_devices('GPU'))

# Detaillierte Info
from tensorflow.python.client import device_lib
print("\nAlle Geräte:")
print(device_lib.list_local_devices())

# Teste ob GPU funktioniert
if tf.config.list_physical_devices('GPU'):
    print("\n✅ GPU wird erkannt!")
    with tf.device('/GPU:0'):
        # Ein einfacher Test auf der GPU
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
        c = tf.matmul(a, b)
        print("GPU Test erfolgreich:", c.numpy())
else:
    print("\n❌ GPU wird nicht erkannt")
    print("Mögliche Lösungen:")
    print("1. CUDA Toolkit installieren")
    print("2. cuDNN installieren")
    print("3. TensorFlow-GPU Version installieren")

Python Version: 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:39:05) [MSC v.1929 64 bit (AMD64)]
TensorFlow Version: 2.10.0
CUDA verfügbar: False
GPUs erkannt: []

Alle Geräte:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15497296952634672881
xla_global_id: -1
]

❌ GPU wird nicht erkannt
Mögliche Lösungen:
1. CUDA Toolkit installieren
2. cuDNN installieren
3. TensorFlow-GPU Version installieren


In [ ]:
def load_all_data():
    pfad = config.PROCESSED_DATA_DIR 
    main_path = Path(pfad)
    
    
    # alles laden und seperat halten 
    # all_image_paths = []
    # all_power = pd.DataFrame()
    # all_stats = pd.DataFrame()
    # for session in main_path.iterdir():
    #     if session.is_dir():
    #         for color in session.iterdir():
    #             if color.is_dir():
    #                 for image_file in color.glob("*.*"):
    #                     if image_file.suffix.lower() in ['.png']:
    #                         all_image_paths.append(str(image_file))
    #             elif color.is_file() and color.name == "power.csv":
    #                 df = pd.read_csv(color)
    #                 all_power = pd.concat([all_power, df], ignore_index=True)
    #             elif color.is_file() and color.name == "stats.csv":
    #                 df = pd.read_csv(color)
    #                 all_stats = pd.concat([all_stats, df], ignore_index=True)
    
    all_image_paths = []
    all_power = pd.DataFrame()
    all_stats = pd.DataFrame()
    all_information = pd.DataFrame()
    for session in main_path.iterdir():
        if session.is_dir():
            farb_liste = []
            for color in session.iterdir():
                if color.is_file() and color.name == "power.csv":
                    all_power = pd.read_csv(color)
                elif color.is_file() and color.name == "stats.csv":
                    all_stats = pd.read_csv(color)
                else:
                    if color.name != "subBackground":
                        farb_liste.append(color.name)
                
            data = pd.DataFrame()
            
            i = 0
            time_temp = 0
            time_list = []
            pfad_list = []
            stats_list = []
            power_list = []
            while i < len(all_power):
                frame = "unkown"
                for col in all_power.columns:
                    frame = all_power["Frame"].iloc[i]
                    time_temp = all_power["time"].iloc[i]
                    if col == "Frame":
                        continue
                    elif col == "time":
                        continue
                    elif col not in farb_liste:
                        continue
                    else:    
                        pfad_list.append(f"{str(session)}\{col}\{frame}.png")
                        time_list.append(time_temp)
                        if col in all_stats.columns.values:
                            stats_list.append(all_stats[col].iloc[i])
                        else:
                            stats_list.append(-1)
                            
                        if col in all_power.columns.values:
                            power_list.append(all_power[col].iloc[i])
                        else:
                            power_list.append(-1)

                        
                i += 1
            data = pd.DataFrame({
                "path": pfad_list,
                "time": time_list,
                "power": power_list,
                "area": stats_list
            })
            

            all_information = pd.concat([all_information, data], ignore_index=True)           
              
            
                      
            # for color in session.iterdir():
            #     if color.is_dir():
            #         for image_file in color.glob("*.*"):
            #             if image_file.suffix.lower() in ['.png']:
            #                 all_image_paths.append(str(image_file))
            #     elif color.is_file() and color.name == "power.csv":
            #         df = pd.read_csv(color)
            #         all_power = pd.concat([all_power, df], ignore_index=True)
            #     elif color.is_file() and color.name == "stats.csv":
            #         df = pd.read_csv(color)
            #         all_stats = pd.concat([all_stats, df], ignore_index=True)


    all_information = all_information.dropna(subset=["power"])
    return all_information
dataset = load_all_data()


# ==== Train test split ====
train_df, test_df = train_test_split(
    dataset, 
    test_size=0.2,       
    random_state=42,     
    shuffle=True    
)
print(f"Trainingsdaten: {len(train_df)}")
print(f"Testdaten: {len(test_df)}")

def create_data_generator(dataframe, batch_size=32, target_size=(1920, 1080)):
    """Erstellt einen Generator der Bilder batchweise lädt"""
    datagen = ImageDataGenerator(
        rescale=1./255  # Normalisierung
    )
    
    generator = datagen.flow_from_dataframe(
        dataframe=dataframe,
        x_col='path',      # Spalte mit Bildpfaden
        y_col='power',     # Spalte mit Target values
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',  # 'raw' für Regression (nicht classification)
        shuffle=True,
        workers=10,
        max_queue_size=10
    )
    return generator


#dataset.to_csv("missing_paths.csv", index=False)
#dataset["path_exists"] = dataset["path"].apply(lambda x: Path(x).exists())



Trainingsdaten: 2888
Testdaten: 723


In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint

batch_size = 10
x_size = 512 #1920
y_size = 512 #1080


# Einfacher Autoencoder
train_generator = create_data_generator(train_df, batch_size=batch_size, target_size=(x_size, y_size))
test_generator = create_data_generator(test_df, batch_size=batch_size, target_size=(x_size, y_size))
def build_model(input_shape=(1920, 1080, 3)):
    # Encoder
    with tf.device('/device:GPU:0'):  
        model = models.Sequential([
            # Erster Convolutional Block
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            layers.MaxPooling2D((2, 2)),
            
            # Zweiter Convolutional Block
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            
            # Dritter Convolutional Block
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            
            # Flatten und Dense Layer
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.4),  # Erhöhte Dropout-Rate für bessere Regularisierung
            layers.Dense(32, activation='relu'),
            layers.Dense(1)  # Regression output
        ])
        
        # Optimizer mit angepassten Parametern
        def rmsle(y_true, y_pred):
            y_true = tf.cast(y_true, tf.float32)  # Konvertiere y_true zu float32
            y_pred = tf.cast(y_pred, tf.float32)  # Konvertiere y_pred zu float32
            y_true = tf.math.maximum(y_true, 1)  # Verhindert, dass y_true 0 wird
            y_pred = tf.math.maximum(y_pred, 1) 
            return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Angepasste Lernrate
            loss=rmsle,
            metrics=["mae"]
        )
    return model

# Autoencoder erstellen und trainieren
model = build_model(input_shape=(x_size, y_size, 3))
model.summary()

# Training
# Callback speichern des besten Modells
checkpoint = ModelCheckpoint(
    filepath="best_model1.h5", 
    monitor="val_loss",       
    save_best_only=True,     
    verbose=1
)
with tf.device('/device:GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_df) // batch_size,
        epochs=30,
        validation_data=test_generator,
        validation_steps=len(test_df) // batch_size,
        verbose=1
    )

Found 2631 validated image filenames.
Found 640 validated image filenames.


c:\Users\amonp\miniconda3\envs\MachineLearning_SC\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 257 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
c:\Users\amonp\miniconda3\envs\MachineLearning_SC\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 83 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
c:\Users\amonp\miniconda3\envs\MachineLearning_SC\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_31 (Conv2D)              │ (None, 1918, 1078, 32) │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 959, 539, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 957, 537, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 478, 268, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4099328)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │   131,178,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,189,217 (500.45 MB)

 Trainable params: 131,189,217 (500.45 MB)

 Non-trainable params: 0 (0.00 B)

c:\Users\amonp\miniconda3\envs\MachineLearning_SC\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 2/90 ━━━━━━━━━━━━━━━━━━━━ 3:53 3s/step - loss: 282705821696.0000 - mae: 330684.0000

KeyboardInterrupt: 

In [ ]:

test_loss, test_mae = model.evaluate(test_generator, verbose=1)
print(f"\nTest Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

mean_power = dataset["power"].mean()
print(f"Mean of dataset: {mean_power}")
median_power = dataset["power"].median()
print(f"Median of dataset: {median_power}")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training mae')
plt.plot(history.history['val_mae'], label='Validation mae')
plt.title('Model mae')
plt.xlabel('Epoch')
plt.ylabel('mae')
plt.legend()

plt.tight_layout()
plt.savefig("training_model1.png")
plt.show()